<a href="https://colab.research.google.com/github/FJPO/citation_predictor/blob/main/Internship_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PATH = input('Введите путь: ')

In [3]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from time import gmtime, strftime

In [4]:
# Здесь модель создается, обучается и тестируется
def testNewModel(x_data, y_data):
  x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size = 0.1)

  model = Sequential()
  model.add(Dense(200, input_dim = x_train.shape[1], activation = 'relu'))
  model.add(Dense(200, activation = 'relu'))
  model.add(Dense(200, activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation = 'linear'))
  model.compile(optimizer = Adam(lr = 0.001), loss = 'mae')

  log = model.fit(x_train, y_train, 16, 40, verbose=0, validation_split = 0.1)

  y_test_from_model = yScaler.inverse_transform(model.predict(x_test))
  y_actual = yScaler.inverse_transform(y_test)

  return {'log' : log,
          'r2_score' : metrics.r2_score(y_test, model.predict(x_test)),
          'model' : model
          }

# Метод для сохранения графика ошибки
def saveLossPlot(result, descr):
  if(descr == ''): descr = 'Plot'
  # print('Точность:', result['r2_score'])
  log = result['log']
  plt.plot(log.history['loss'], label='Ошибка на обучающем наборе')
  plt.plot(log.history['val_loss'], label='Ошибка на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Ошибка')
  plt.legend()

  plt.title(descr)
  plt.savefig(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + '__' + descr + '.png')
  # plt.show()

In [ ]:
data = pd.read_csv(PATH)
data = data.drop(['authors_mean_pagerank', 'authors_mean_productivity', 'journal_pagerank'], axis = 1)
data = data[np.logical_and(~np.isnan(data).any(axis = 1), data['c5'] < 100)]


# Предобработка данных
yScaler = preprocessing.MinMaxScaler()
y_data = yScaler.fit_transform(
    np.array(data['c5']).reshape(-1,1)
    )

x_data = data.drop('c5', axis = 1)
for i, name in enumerate(x_data.columns):
    x_data[name] = preprocessing.MinMaxScaler().fit_transform(np.array(data[name]).reshape(-1, 1))


In [ ]:
print('R^2 модели - ', testNewModel(x_data, y_data))
print('Данный результат не является точным, для минимизации разброса заустите код из ячейки ниже')

In [ ]:
#
#
# Здесь вычисляется средняя и наибольшаяя точность работы модели
# 
#
results = []
for i in range(50):
  if(i%5 == 0): print(int(i/5), "-", sep = '', end = '')
  model_result = testNewModel(x_data, y_data)
  results.append(model_result['r2_score'])
  if(model_result['r2_score'] > 0.55):
    model_result['model'].save('r2_' + str(round(model_result['r2_score'], 3)) + '.h5')
    saveLossPlot(model_result, 'r2_' + str(round(model_result['r2_score'], 3)))
  
print('\nТочность в среднем:', np.array(results).mean())
print('Точность в лучшем случае:', np.array(results).max())

In [69]:
# Код вычисляет зависимость работы модели от разных параметров

for i, name in enumerate(x_data.columns):
  results = []
  for int in range(50):
    model = Sequential()
    model.add(Dense(40, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(40, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))

    model.compile(optimizer = Adam(lr = 0.001), loss = 'mae')

    log = model.fit(x_train, y_train, 16, 20, verbose=0, validation_split = 0.1)
    results.append(metrics.r2_score(y_test, model.predict(x_test)))
  

  print('\nВ среднем случае для', name, ':\t', np.array(results).mean())
  print('В лучшем случае для', name, ':\t', np.array(results).max())




В среднем случае для recency :	 0.09454302118091251
В лучшем случае для recency :	 0.37379053139190144

В среднем случае для topic_rank :	 0.09096684243741607
В лучшем случае для topic_rank :	 0.36946172504362795

В среднем случае для diversity :	 0.11770783352169037
В лучшем случае для diversity :	 0.3737220721830494

В среднем случае для authors_mean_rank :	 0.06826653363483065
В лучшем случае для authors_mean_rank :	 0.3816612277779833

В среднем случае для authors_mean_hindex :	 0.08999349676235925
В лучшем случае для authors_mean_hindex :	 0.2901572128852693

В среднем случае для authors_mean_gindex :	 0.12432858358774192
В лучшем случае для authors_mean_gindex :	 0.3693337902446434

В среднем случае для authors_mean_sociality :	 0.093443662414163
В лучшем случае для authors_mean_sociality :	 0.32546686865706487

В среднем случае для journal_rank :	 0.05419405385924294
В лучшем случае для journal_rank :	 0.3073566772018187

В среднем случае для title_len :	 0.09284322321074417
В 